In [1]:
import tsa_utils

from shutil import copytree
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
sys.path.append(os.getcwd())
import pip
pip.main(['install', '--upgrade', 'dask'])
import sklearn.decomposition
from sklearn.preprocessing import StandardScaler
import pickle
import scipy.misc
import skimage.io
import cv2
import skimage.measure
import pickle

Requirement already up-to-date: dask in /home/cody/anaconda3/envs/python3/lib/python3.6/site-packages


In [2]:
#Model parameters
BATCH_SIZE=10
FILTER_COUNT=12
KERNEL_SIZE1=(1,6,6)
DEPTHSTRIDE=1
XSTRIDE=1
YSTRIDE=1
POOLSIZE1=(16,3,3)
POOLSIZE2=(3,3)
POOL_STRIDES1=(1,2,2)
POOL_STRIDES=(1,1)
STEPS=3000
XSIZE=256
YSIZE=330
LEARNING_RATE=0.001
EPSILON=0
IMAGE_DEPTH=16
CHANNELS=1
FLAT_POOL_SIZE=20828

DATA_PATH="./images/"
CHECKPOINT_PATH="/output/multi_class"
MODEL_ID="pca4"

tf.logging.set_verbosity(tf.logging.INFO)
pca_model = pickle.load(open("../tsa_data/pca_models/medium_safe_pca.p","rb+"))


/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator IncrementalPCA from version 0.18.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
class SingleClassPcaZoneModel():

    def __init__(self, model_id, ids, zone, data_path, labels, pca_model, checkpoint_path=".", localize=False):
        self.model_id = model_id
        self.ids = ids

        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.labels = labels
        self.localize = localize
        self.pca_model=pca_model

    def build_model(self, data, labels, mode):
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=10
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        print(data)
        pool1 = tf.layers.max_pooling3d(inputs=data, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool1")
        print(pool1)
        flat_pool = tf.reshape(pool1, [BATCH_SIZE, FLAT_POOL_SIZE])
        flat_pool=tf.identity(flat_pool, name="flat_pool")
        print(flat_pool)
        logits = tf.layers.dense(inputs=flat_pool, units=2, trainable=True)
        logits = tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,2])
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.softmax(
              logits, name="prob_tensor")}

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            flat_labels = tf.reshape(labels, [BATCH_SIZE, 2])
            test_labels=tf.identity(flat_labels, name="labels")
            loss = tf.losses.softmax_cross_entropy(onehot_labels=test_labels, logits=logits)
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    def train_model(self, tensors_to_log, reuse=False):
        if reuse:
            tsa_classifier = self.model
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                     model_dir=self.checkpoint_path + "/" + self.model_id)
        logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
        tsa_classifier.fit(
            x= tsa_utils.ResidueIterator(self.ids, self.data_path, self.pca_model), 
            y=tsa_utils.InputLabelsIterator(self.ids, self.labels), 
            steps=STEPS, 
            batch_size=BATCH_SIZE, 
            monitors=[logging_hook])
        self.model = tsa_classifier

    def load_model(self):
        print(self.checkpoint_path, self.model_id)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id)
        print ("LOADED MODEL AT STEP " + str(tsa_classifier.get_variable_value("global_step")) + " FROM " + self.checkpoint_path + "/" + self.model_id )
        self.model = tsa_classifier

    def bootstrap_model(self, path=None):
        if path:
            checkpoint_path = path
        else:
            checkpoint_path = self.checkpoint_path + "/" + self.model_id
        print ("USING CHECKPOINT PATH  " + checkpoint_path)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=checkpoint_path)
        print ("BOOTSTRAPPED AT STEP " + str(tsa_classifier.get_variable_value("global_step")))
        self.model = tsa_classifier 

    def predict(self):
        return self.model.predict(x=tsa_utils.ResidueIterator(self.ids, self.data_path, self.pca_model, repeating=False))

In [4]:
class MultiClassPcaZoneModel():
    def __init__(self, model_id, ids, data_path, labels, pca_model, checkpoint_path=".", localize=False):
        self.model_id = model_id
        self.ids = ids

        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.labels = labels
        self.localize = localize
        self.pca_model=pca_model

    def build_model(self, data, labels, mode):
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=10
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        print(data)
        pool1 = tf.layers.max_pooling3d(inputs=data, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool1")
        print(pool1)
        flat_pool = tf.reshape(pool1, [BATCH_SIZE, FLAT_POOL_SIZE])
        flat_pool=tf.identity(flat_pool, name="flat_pool")
        print(flat_pool)
        logits = tf.layers.dense(inputs=flat_pool, units=17, trainable=True)
        logits = tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,17])
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.softmax(
              logits, name="prob_tensor")}

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            flat_labels = tf.reshape(labels, [BATCH_SIZE, 17])
            test_labels=tf.identity(flat_labels, name="labels")
            loss = tf.losses.softmax_cross_entropy(onehot_labels=test_labels, logits=logits)
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    def train_model(self, tensors_to_log, reuse=False):
        if reuse:
            tsa_classifier = self.model
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                     model_dir=self.checkpoint_path + "/" + self.model_id)
        logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
        tsa_classifier.fit(
            x= tsa_utils.ResidueIterator(self.ids, self.data_path, self.pca_model), 
            y=tsa_utils.InputLabelsIterator(self.ids, self.labels), 
            steps=STEPS, 
            batch_size=BATCH_SIZE, 
            monitors=[logging_hook])
        self.model = tsa_classifier

    def load_model(self):
        print(self.checkpoint_path, self.model_id)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id)
        print ("LOADED MODEL AT STEP " + str(tsa_classifier.get_variable_value("global_step")) + " FROM " + self.checkpoint_path + "/" + self.model_id )
        self.model = tsa_classifier

    def bootstrap_model(self, path=None):
        if path:
            checkpoint_path = path
        else:
            checkpoint_path = self.checkpoint_path + "/" + self.model_id
        print ("USING CHECKPOINT PATH  " + checkpoint_path)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=checkpoint_path)
        print ("BOOTSTRAPPED AT STEP " + str(tsa_classifier.get_variable_value("global_step")))
        self.model = tsa_classifier 

    def predict(self):
        return self.model.predict(x=tsa_utils.ResidueIterator(self.ids, self.data_path, self.pca_model, repeating=False))

# Fit Single-Class Model

In [ ]:
tensors_to_log =  {"probabilities": "prob_tensor",
                    "actual":"labels"}
image_df = pd.read_csv(DATA_PATH + '/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()
# image_df = image_df[image_df["zone"]=="Zone14"]
# image_df=image_df[image_df["Probability"] >0]

ids = image_df["id"].unique()
ids.sort()
training_ids = ids

labels = image_df[image_df['id'].isin(training_ids)]
labels = labels.sort_values("id")



with tf.Session() as sess:
    for zone in range(14,15):
        print("FITTING MODEL FOR ZONE " + str(zone))
        train_labels = labels[labels["zone"]=="Zone"+str(zone)]
        train_labels["class0"] = 0
        train_labels["class1"] = 0
        train_labels.loc[train_labels['Probability'] == 0, 'class0'] = 1
        train_labels.loc[train_labels['Probability'] == 1, 'class1'] = 1
        train_labels = np.reshape(np.array(train_labels[["class0","class1"]]), [-1,2])
        #model = ZoneModel(MODEL_ID, training_ids, "Zone" + str(zone), slice_locations[zone][1], slice_locations[zone][0], DATA_PATH, train_labels, CHECKPOINT_PATH, localize=False)
        with tf.Session() as sess:
            model = PcaZoneModel(MODEL_ID, ids, zone, DATA_PATH, train_labels, pca_model, checkpoint_path=CHECKPOINT_PATH, localize=False)     
            model.train_model(tensors_to_log, reuse=False)

# Fit Multi-Class Model

In [5]:
tensors_to_log =  {"probabilities": "prob_tensor",
                    "actual":"labels"}
image_df = pd.read_csv(DATA_PATH + '/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_Zone", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()
ids = image_df["id"].unique()
ids.sort()
training_ids = ids
labels = image_df
labels = labels.sort_values("id")

labels.zone = labels.zone.astype("int")
labels = labels.sort_values(["id","zone"])
labels = labels["Probability"]
labels = np.reshape(labels, (-1,17))

with tf.Session() as sess:
    #model = ZoneModel(MODEL_ID, training_ids, "Zone" + str(zone), slice_locations[zone][1], slice_locations[zone][0], DATA_PATH, train_labels, CHECKPOINT_PATH, localize=False)
    with tf.Session() as sess:
        model = MultiClassPcaZoneModel(MODEL_ID, ids, DATA_PATH, labels, pca_model, checkpoint_path=CHECKPOINT_PATH, localize=False)     
        model.train_model(tensors_to_log, reuse=False)

/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1e91e4c898>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/output/multi_class/pca4'}
Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1e91e4c898>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config':

From <ipython-input-4-5eba8fa8a6e9>:58: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


From <ipython-input-4-5eba8fa8a6e9>:58: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with y is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


...

/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/f49798fea19cafb93d31109d7bfce948data.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/f4c06c10ec1e5a4ef33018f2febd8d2edata.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/f5044f179fdbce9ae3f5da283babd359data.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/f553dc398064f86e9bb3fa06895aabecdata.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./outp

INFO:tensorflow:Saving checkpoints for 432 into /output/multi_class/pca4/model.ckpt.
Saving checkpoints for 432 into /output/multi_class/pca4/model.ckpt.


/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/fdb996a779e5d65d043eaa160ec2f09fdata.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/ff4943024cb698b2258abd66fa6d6d62data.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/ff8d4d101cc5fb16bb06356f0e5f76a8data.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/cody/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./output_images/ff9c9b7de5dacc8315e2bbc18c451c49data.png is a low contrast image
  warn('%s is a low contrast image' % fname)


INFO:tensorflow:Saving checkpoints for 471 into /output/multi_class/pca4/model.ckpt.
Saving checkpoints for 471 into /output/multi_class/pca4/model.ckpt.


KeyboardInterrupt: 

# Eval Single-Class Model

In [ ]:
image_df = pd.read_csv(DATA_PATH + 'stage1_sample_submission.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()
labels = image_df[image_df['id'].isin(ids)]
all_labels = labels.sort_values("id")
csv_file = open(CHECKPOINT_PATH + "/submission.csv", "w+")
csv_file.write("Id,Probability\n")
for zone in range(1,18):
    print("EVALUATING MODEL FOR ZONE " + str(zone))
    labels = all_labels[all_labels["zone"]=="Zone"+str(zone)]
    labels["class0"] = 0
    labels["class1"] = 0
    labels.loc[labels['Probability'] == 0, 'class0'] = 1
    labels.loc[labels['Probability'] == 1, 'class1'] = 1
    labels = np.reshape(np.array(labels[["class0","class1"]]), [-1,2])
    print(len(labels))
    model = PcaZoneModel(MODEL_ID, ids, zone, DATA_PATH, labels, pca_model, checkpoint_path=CHECKPOINT_PATH, localize=False)     
    model.load_model()
    print ("LOADED MODEL " + str(model))
    predicted = np.array([x["probabilities"][1] for x in list(model.predict())])
    for i, prediction in enumerate(predicted):
        csv_file.write(str(ids[i]) + "_Zone" + str(zone) + "," + str(prediction) + "\n")
csv_file.close()